In [1]:
import os
import numpy as np
import nibabel as nib
import SimpleITK as sitk
from scipy import ndimage as ndi
import pywt
import six
from six.moves import range

### LoG

In [2]:
def getLoGImage(inputImage, sigmaValues):
   
    result = {}
    size = np.array(inputImage.shape)
    spacing = np.array(inputImage.header.get_zooms())

    if np.min(size) < 4:
        print('Image too small to apply LoG filter.')s
        return result

    for sigma in sigmaValues:
#         print(f"Computing LoG with sigma {sigma}")
        if sigma > 0.0:
            if np.all(size >= np.ceil(sigma / spacing) + 1):
                img_data = inputImage.get_fdata()
                smoothed_data = ndi.gaussian_filter(img_data, sigma)
                laplacian_data = ndi.laplace(smoothed_data)
                inputImageName = f"log-sigma-{str(sigma).replace('.', '-')}mm-3D"
                im = nib.Nifti1Image(laplacian_data, affine=inputImage.affine, header=inputImage.header)
                result[inputImageName] = im
            else:
                print("Sigma/spacing + 1 must be greater than the size of the inputImage.")
        else:
            print("Sigma must be greater than 0.0.")
    return result

### Wavelet

In [5]:
def getWaveletImage(inputImage): 
    Nd = len(inputImage.shape)
    axes = list(range(Nd - 1, -1, -1))
    
    approx, ret = _swt3(inputImage, tuple(axes))
    
    for idx, wl in enumerate(ret, start=1):
        for decompositionName, decompositionImage in wl.items():
            if idx == 1:
                inputImageName = 'wavelet-%s' % (decompositionName)
            else:
                inputImageName = 'wavelet%s-%s' % (idx, decompositionName)
            yield decompositionImage, inputImageName
            
    if len(ret) == 1:
        inputImageName = 'wavelet-%s' % ('L' * len(axes))
    else:
        inputImageName = 'wavelet%s-%s' % (len(ret), ('L' * len(axes)))
        
    yield approx, inputImageName


def _swt3(inputImage, axes):  # Stationary Wavelet Transform 3D
    wavelet =  'coif1'
    level = 1
    start_level = 0

    matrix = inputImage.get_fdata()  # This function gets a numpy array from the SimpleITK Image "inputImage"
    matrix = np.asarray(matrix) # The function np.asarray converts "matrix" (which could be also a tuple) into an array.

    original_shape = matrix.shape
    padding = tuple([(0, 1 if dim % 2 != 0 else 0) for dim in original_shape])
    data = matrix.copy()  
    data = np.pad(data, padding, 'wrap') 

    if not isinstance(wavelet, pywt.Wavelet):
        wavelet = pywt.Wavelet(wavelet)

    for i in range(0, start_level): 
        dec = pywt.swtn(data, wavelet, level=1, start_level=0, axes=axes)[0]
        # copies in "data" just the "aaa" decomposition (i.e. approximation; No of consecutive 'a's = len(axes))
        data = dec['a' * len(axes)].copy()

    ret = []  # initialize empty list
    for i in range(start_level, start_level + level):
        dec = pywt.swtn(data, wavelet, level=1, start_level=0, axes=axes)[0]
        data = dec['a' * len(axes)].copy()
        
        dec_im = {}  # initialize empty dict
        for decName, decImage in six.iteritems(dec):
            if decName == 'a' * len(axes):
                continue
            decTemp = decImage.copy()
            decTemp = decTemp[tuple(slice(None, -1 if dim % 2 != 0 else None) for dim in original_shape)]
            dec_img = nib.Nifti1Image(decTemp, inputImage.affine)
            dec_im[str(decName).replace('a', 'L').replace('d', 'H')] = dec_img
    
        ret.append(dec_im) 

    data = data[tuple(slice(None, -1 if dim % 2 != 0 else None) for dim in original_shape)]
    approximation = nib.Nifti1Image(data, inputImage.affine)
    return approximation, ret  

### squre

In [8]:
def getSquareImage(inputImage):

    im = inputImage.get_fdata()
    im = im.astype('float64')
    coeff = 1 / np.sqrt(np.max(np.abs(im)))
    im = (coeff * im) ** 2
    im = nib.Nifti1Image(im, affine=inputImage.affine, header=inputImage.header)

    yield im, 'square'

### squreRoot

In [11]:
def getSquareRootImage(inputImage):
    
    im = inputImage.get_fdata()
    im = im.astype('float64')
    coeff = np.max(np.abs(im))
    im[im > 0] = np.sqrt(im[im > 0] * coeff)
    im[im < 0] = - np.sqrt(-im[im < 0] * coeff)
    im = nib.Nifti1Image(im, affine=inputImage.affine, header=inputImage.header)

    yield im, 'squareroot'

### logarithm

In [14]:
def getLogarithmImage(inputImage):
   
    
    im = inputImage.get_fdata()
    im = im.astype('float64')
    im_max = np.max(np.abs(im))
    im[im > 0] = np.log(im[im > 0] + 1)
    im[im < 0] = - np.log(- (im[im < 0] - 1))
    im = im * (im_max / np.max(np.abs(im)))
    im = nib.Nifti1Image(im, affine=inputImage.affine, header=inputImage.header)

    yield im, 'logarithm'

In [15]:
# logarithmImage = getLogarithmImage(nii_obj)

In [16]:
# # 处理图像并按需打印输出结果
# for im, label in logarithmImage:
#     print(f'Result type: {label}, dtype: {im.get_fdata().dtype}, shape: {im.get_fdata().shape}')
#     print(f'Intensity range: [{np.min(im.get_fdata())}, {np.max(im.get_fdata())}]\n')

### exponential

In [17]:
def getExponentialImage(inputImage):
       
    im = inputImage.get_fdata()
    im = im.astype('float64')
    im_max = np.max(np.abs(im))
    coeff = np.log(im_max) / im_max
    im = np.exp(coeff * im)
    im = nib.Nifti1Image(im, affine=inputImage.affine, header=inputImage.header)

    yield im, 'exponential'

### gradient

In [20]:
def getGradientImage(inputImage):
        
    sitkImage = sitk.GetImageFromArray(inputImage.get_fdata())
    sitkImage.CopyInformation(sitk.GetImageFromArray(np.zeros(inputImage.shape)))

    gmif = sitk.GradientMagnitudeImageFilter()
    gmif.SetUseImageSpacing(True)
    im = gmif.Execute(sitkImage)

    im = sitk.GetArrayFromImage(im)
    im = nib.Nifti1Image(im, affine=inputImage.affine, header=inputImage.header)
    
    yield im, 'gradient'


In [23]:
def apply_all_transforms(data_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(data_folder):
        if filename.endswith('.nii.gz'):
            nii_obj = nib.load(os.path.join(data_folder, filename))

            # LoG transform
            sigmas = [1.0, 2.0, 3.0, 4.0, 5.0]
            result_dict = getLoGImage(nii_obj, sigmas)
            for key, data in result_dict.items():
                LoG_filename = f"{os.path.splitext(filename)[0][:10]}_{key}.nii.gz"
                save_nii_file(data, LoG_filename, output_folder)
              
            # Wavelet transform
            waveletGenerator = getWaveletImage(nii_obj)
            for image, name in waveletGenerator:
                wavelet_filename = f"{os.path.splitext(filename)[0][:10]}_{name}.nii.gz"
                save_nii_file(image, wavelet_filename, output_folder)
              
            # Square, Square Root, Logarithm, Exponential and Gradient transforms
            squareImage = getSquareImage(nii_obj)
            for im, label in squareImage:
                square_filename = f"{os.path.splitext(filename)[0][:10]}_{label}.nii.gz"
                save_nii_file(im, square_filename, output_folder)
                
            squareRootImage = getSquareRootImage(nii_obj)
            for im, label in squareRootImage:
                square_root_filename = f"{os.path.splitext(filename)[0][:10]}_{label}.nii.gz"
                save_nii_file(im, square_root_filename, output_folder)
            
            logarithmImage = getLogarithmImage(nii_obj)
            for im, label in logarithmImage:
                logarithm_filename = f"{os.path.splitext(filename)[0][:10]}_{label}.nii.gz"
                save_nii_file(im, logarithm_filename, output_folder)
            
            exponentialImage = getExponentialImage(nii_obj)
            for im, label in exponentialImage:
                exponential_filename = f"{os.path.splitext(filename)[0][:10]}_{label}.nii.gz"
                save_nii_file(im, exponential_filename, output_folder)
                
            gradientImage = getGradientImage(nii_obj)
            for im, label in gradientImage:
                gradient_filename = f"{os.path.splitext(filename)[0][:10]}_{label}.nii.gz"
                save_nii_file(im, gradient_filename, output_folder)

def save_nii_file(data, filename, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    img = data
    output_path = os.path.join(output_folder, filename)
    nib.save(img, output_path)


data_folders = ['./path/to/files']
output_folders = ['./save_path/']

for data_folder, output_folder in zip(data_folders, output_folders):
    apply_all_transforms(data_folder, output_folder)


Sigma/spacing + 1 must be greater than the size of the inputImage.
